#  Nearest Neighbour item based collabarative filtering 

In [6]:
import pandas as pd
import numpy as np

## movie lens dataset 
#https://grouplens.org/datasets/movielens/latest/


In [8]:
# reading only required colunmns from the csv

ratings_df = pd.read_csv('C:/Users/shash/OneDrive/Documents/MachineLearning/Recommendation/movie_lens_small_dataset/ratings.csv',
                        usecols=['userId', 'movieId', 'rating'],
                        dtype = {'moviId' : 'int32', 'title' : 'str'});
movies_df = pd.read_csv('C:/Users/shash/OneDrive/Documents/MachineLearning/Recommendation/movie_lens_small_dataset/movies.csv',
                       usecols = ['movieId', 'title'],
                       dtype = {'userID':'int32', 'movieId':'int32', 'rating':'float32'});

In [9]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [16]:
merged_df = pd.merge(movies_df, ratings_df, on="movieId")
merged_df.head()
merged_df.shape

(100836, 4)

In [17]:
# remove NaN in title column
combined_movie_rating = merged_df.dropna(axis=0, subset=['title'])
combined_movie_rating

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [26]:
# find total number of ratings for each movie

movie_rating_count = (combined_movie_rating.groupby(by = ['title'])['rating'].
                      count().
                      reset_index().
                      rename(columns= {'rating':'totalRatingCount'})
                     )

movie_rating_count.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [30]:
# merge rating count with movies df

movies_with_ratingCount = combined_movie_rating.merge(movie_rating_count,on='title', how="left")
movies_with_ratingCount.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [31]:
movies_with_ratingCount.shape

(100836, 5)

In [35]:
# filter popular movies
popularity_threshold = 50
# @ symbol is used to read local variables
popular_movies = movies_with_ratingCount.query('totalRatingCount >= @popularity_threshold')

popular_movies.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [36]:
popular_movies.shape

(41362, 5)

In [41]:
# construct pivot table of rating of each user to all the movies
movies_pivot_df = popular_movies.pivot_table(index = 'title', columns='userId', values='rating').fillna(0)
movies_pivot_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [45]:
# coverting pivot table to a array matrix
# for this, we will use csr_matrix function

from scipy.sparse import csr_matrix

movies_features_in_matrix = csr_matrix(movies_pivot_df.values)

movies_features_in_matrix

<450x606 sparse matrix of type '<class 'numpy.float64'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [46]:
# import nearest neighbors

from sklearn.neighbors import NearestNeighbors

knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

knn_model.fit(movies_features_in_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [117]:
movies_pivot_df.shape

(450, 606)

In [118]:
total_number_of_movies = movies_pivot_df.shape[0]
total_number_of_movies

450

In [119]:
# choose any one random number from 0 - 450
index = np.random.choice(total_number_of_movies)
index

0

In [120]:
# get all the ratings of selected movie
randomly_selected_movie_ratings = movies_pivot_df.iloc[index,:].values.reshape(1,-1)

In [121]:
# filter similar movies with a trained knn_model, choosing 6 neighbors

distances, indices = knn_model.kneighbors(randomly_selected_movie_ratings, n_neighbors = 6)

In [124]:
# distances - distance between selected movie and recom movie
# indices - index of movie selected to recom

for i in range(0, len(distances.flatten())):
    if i == 0:
        # it is the selected movie itself
        print("Recommendation for : {0}".format(movies_pivot_df.index[index]))
    else:
        print("Recommended movies : ")
        print("{0}. {1} with distance {2}".format(i,
                                                  movies_pivot_df.index[indices.flatten()[i]],
                                                 distances.flatten()[i]))

Recommendation for : 10 Things I Hate About You (1999)
Recommended movies : 
1. Wedding Singer, The (1998) with distance 0.55065744443742
Recommended movies : 
2. Grease (1978) with distance 0.5718006830472175
Recommended movies : 
3. Bridget Jones's Diary (2001) with distance 0.576185615823046
Recommended movies : 
4. Legally Blonde (2001) with distance 0.595094386344132
Recommended movies : 
5. Clueless (1995) with distance 0.6036181743179048


# Function to recommend based on movie name

In [136]:
def get_recommendation(title):
    movie_rating_info = movies_pivot_df[movies_pivot_df.index == title].values.reshape(1,-1)
    distances, indices = knn_model.kneighbors(movie_rating_info, n_neighbors = 6)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            # it is the selected movie itself
            print("Recommendation for : {0}".format(movies_pivot_df.index[index]))
        else:
            print("Recommended movies : ")
            print("{0}. {1} with distance {2}".format(i,
                                                      movies_pivot_df.index[indices.flatten()[i]],
                                                     distances.flatten()[i]))

In [137]:
get_recommendation('Mystic River (2003)')

Recommendation for : 10 Things I Hate About You (1999)
Recommended movies : 
1. Kill Bill: Vol. 1 (2003) with distance 0.5074114793878831
Recommended movies : 
2. Donnie Brasco (1997) with distance 0.5095515589144293
Recommended movies : 
3. Million Dollar Baby (2004) with distance 0.5117906540071877
Recommended movies : 
4. Departed, The (2006) with distance 0.5262978591830616
Recommended movies : 
5. Traffic (2000) with distance 0.5305264023319554
